# Gráficos  - Prjeto 3 - QTA

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Dados Coletados 

In [ ]:
pH = 0.12
DBO = 0.10
OD = 0.17
temp = 0.10
nitro = 0.10
turb = 0.08
fos = 0.1
res = 0.08
fosf = 0.1
coli = 0.15
list_data = [DBO, nitro, coli, pH, OD, temp, fosf, turb, res]